In [1]:
import copy

# 外皮情報LV2からLV3へのコンバート

## 1. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）
- 住戸全体の外皮（床下空間に属するものを除く）、および、室と床下空間の間の床は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。
- 各部位の熱橋は、対応する一般部のU値の低減に置き換える。一般部の断熱材の厚を調整する。
- その他は、用途別情報が得られている場合と同様に求める。

### 《土間床等面積が0㎡より大きい場合の扱い》
$$ \qquad
A_{b}=min(A_主 + A_他 + A_非, \sum_{i}^{}S_{df,i})
\qquad (1)
\\
$$

- 入力値：
  - 室用途不明の空間に属する土間床等の部位iの面積$[m ^ 2]$：$S_{df,i}$

- 出力値：

  - 室と床下空間の間の床面積の合計$[m ^ 2]$：$A_{b}$

### 《室用途別の床面積に応じた用途別面積の推定》
$$ \qquad
A_{主,i,j}= \frac{A_{i,j}・A_主}{A_主 + A_他 + A_非}, \quad
A_{他,i,j}= \frac{A_{i,j}・A_他}{A_主 + A_他 + A_非}\quad, \quad
A_{非,i,j}= \frac{A_{i,j}・A_非}{A_主 + A_他 + A_非}\quad
\qquad (2),\;(3),\;(4)
\\
$$
$$ \qquad
L_{主,i,j}= \frac{L_{i,j}・A_主}{A_主 + A_他 + A_非}, \quad
L_{他,i,j}= \frac{L_{i,j}・A_他}{A_主 + A_他 + A_非}\quad, \quad
L_{非,i,j}= \frac{L_{i,j}・A_非}{A_主 + A_他 + A_非}\quad
\qquad (5),\;(6),\;(7)
\\
$$
- 入力値：

  - 主居室、その他の居室、非居室の床面積$[m ^ 2]$：$A_主$、$A_他$、$A_非$
  - 外皮の方位j、部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,j}$
  - 熱橋及び土間床等の方位jの外周部𝑖の長さ$[m]$：$L_{i,j}$

- 出力値：

  - 主居室、その他の居室、非居室の外皮の方位j、部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{主,i,j}$、$A_{他,i,j}$、$A_{非,i,j}$
  - 主居室、その他の居室、非居室の熱橋及び土間床等の方位jの外周部𝑖の面積$[m ^ 2]$：$L_{主,i,j}$、$L_{他,i,j}$、$L_{非,i,j}$
    

#### Function

In [2]:
def GetAreaByRoomUse( target_area, main_floor_area, other_floor_area, total_floor_area, space_type ):
    # target_area as double      : 方位j、部位iの面積または長さ[m]
    # total_floor_area as double : 床面積の合計[m]
    # main_floor_area as double  : 主たる居室の床面積[m]
    # other_floor_area as double : その他の居室の床面積[m]
    # space_type as string       : 空間種類　'main' or 'other' or 'nonliving' or 'underfloor'

    if space_type == 'main':
        rate = main_floor_area / total_floor_area
    elif space_type == 'other':
        rate = other_floor_area / total_floor_area
    elif space_type == 'nonliving':
        rate = ( total_floor_area - ( main_floor_area + other_floor_area ) ) / total_floor_area
    else :
        raise ValueError('指定された空間種類は定義されていません')  
    
    return target_area * rate

#### Example

In [3]:
area = GetAreaByRoomUse(50.0, 30.0, 60.0, 120.0, 'main')
area

12.5

## 2. コンバート
### 1) 共通

- 地域の区分、主たる居室の床面積(m<sup>2</sup>)、その他の床面積(m<sup>2</sup>)、床面積の合計(m<sup>2</sup>)、簡易外皮入力フラグの情報は維持する。

#### Input

```
{
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'IsSimplifiedInput'           : # bool値 簡易入力から生成したデータか否かを表す, LV1からの場合は常にTrueとなる。
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)
}
```

#### Output

```
{
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'IsSimplifiedInput'           : # bool値 簡易入力から生成したデータか否かを表す, LV1からの場合は常にTrueとなる。
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)
}
```

#### Function

In [4]:
def convert_common(d):
    return {
        'Region'                     : d['Region'],
        'IsSimplifiedInput'          : d['IsSimplifiedInput'],
        'MainOccupantRoomFloorArea'  : d['MainOccupantRoomFloorArea'],
        'OtherOccupantRoomFloorArea' : d['OtherOccupantRoomFloorArea'],
        'TotalFloorArea'             : d['TotalFloorArea']
    }

#### Example

In [5]:
convert_common({
    'Region' : 6,
    'IsSimplifiedInput': True,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0
})

{'IsSimplifiedInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'Region': 6,
 'TotalFloorArea': 120.0}

### 2) 開口部以外の不透明部位

- 壁体名称、一般部位構造種別、壁体種別、方位、層構成（部材名称、厚さ[m]、熱伝導率[W/mK]、容積比熱[(J/(m<sup>3</sup>・K)]）などの情報は維持する。
- 「壁体が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Input

```
{
    'MainOccupantRoomFloorArea'    : # 主たる居室の床面積(m2),
    'OtherOccupantRoomFloorArea'   : # その他の居室の床面積(m2),
    'TotalFloorArea'               : # 床面積の合計(m2),
    'Walls' : [
    {
        'Name'                     : # 壁体名称
        'Structure'                : # 一般部位構造種別 string値,　以下のいずれか
                                     # 'wood'（木造）, 'rc'（鉄筋コンクリート造等）, 'steel'（鉄骨造）, 'other' (その他/不明)
        'Type'                     : # 壁体種別 string値, 以下のいずれか
                                     # 'ceiling'（屋根・天井）, 'wall'（外壁）, 'floor'（床）, 
                                     # 'BoundaryWall'（界壁）,  'BoundaryFloor'（界床）, 'BoundaryCeiling'（天井（界床））
        'InputMethod_wood'         : # 壁体仕様入力方法（木造） string値, 以下のいずれか
                                     # 'InputUa'（Ua値を入力）, 
                                     # 'InputAllDetails'（断熱部分および熱橋部分の層構成と面積比率を詳細に入力）, 
                                     # 'InputAllLayers'（断熱部分および熱橋部分の層構成のみ入力（面積比率は入力しない））, 
                                     # 'InputUr'（断熱部分の層構成と補正熱貫流率を入力）
                                       'IsSimplifiedInput'がTrueの場合および'Structure'が'wood'以外の場合は無視される。
        'InputMethod_rc'           : # 壁体仕様入力方法（鉄筋コンクリート造） string値, 以下のいずれか
                                     # 'InputUa'（Ua値を入力）, 
                                     # 'InputLayers'（層構成を入力）
                                       'IsSimplifiedInput'がTrueの場合および'Structure'が'rc'以外の場合は無視される。
        'InputMethod_steel'        : # 壁体仕様入力方法（鉄骨造） string値, 以下のいずれか
                                     # 'InputUa'（Ua値を入力）,  
                                     # 'InputUr'（断熱部分の層構成と補正熱貫流率を入力）
                                       'IsSimplifiedInput'がTrueの場合および'Structure'が'steel'以外の場合は無視される。
        'Direction'                : # 方位　string値, 以下のいずれか
                                     # 'top', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'bottom',
                                     # 'ClosedSpace'（外気に通じていない空間）,
                                     # 'OpenBackFloor'（外気に通じる床裏）,
                                     # 'ResidenceSpace'（住戸及び住戸と同様の熱的環境の空間）,
                                     # 'ClosedBackFloor'（外気に通じていない床裏）,
                                     # 'EarthFloor'（土間床等）
        'Area'                     : # 部位面積(m2)
        'Ua'                       : # 平均熱貫流率(W/(m2・K))
                                       'InputMethod_wood'/'InputMethod_rc'/'InputMethod_steel'が
                                       'InputUa'以外の場合は無視される。
        'FloorConstructionMethod'  : # 床の工法種類 string値,　以下のいずれか
                                     # 'Frame_Beam_Joist'（軸組構法・床梁工法（根太間に断熱））,
                                     # 'Frame_Footing_Joist'（軸組構法・束立大引工法（根太間に断熱））,
                                     # 'Frame_Footing_Girder'（軸組構法・束立大引工法（大引間に断熱））,
                                     # 'Frame_Footing_JoistAndGirder'（軸組構法・束立大引工法（根太間及び大引間に断熱））,
                                     # 'Frame_Rigid'（軸組構法・剛床工法）,
                                     # 'Frame_BeamBase_Joist'（軸組構法・床梁土台同面工法（根太間に断熱））,
                                     # 'Wall_Joist'（枠組壁工法（根太間に断熱））
                                       'Type'が'floor'または'BoundaryFloor'で、かつ'InputMethod_wood'が
                                       'InputAllLayers'の場合以外は無視される。
        'WallConstructionMethod'   : # 壁の工法種類 string値, 以下のいずれか
                                     # 'Frame_Normal'（軸組構法・柱、間柱間に断熱）,
                                     # 'Frame_Horizontal'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「横下地」））,
                                     # 'Frame_Vertical'（軸組構法・柱、間柱間に断熱し付加断熱（付加断熱層内熱橋部分が「縦下地」））,
                                     # 'Wall_Normal'（枠組壁工法・たて枠間に断熱）,
                                     # 'Wall_Horizontal'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「横下地」））,
                                     # 'Wall_Vertical'（枠組壁工法・たて枠間に断熱＋付加断熱（付加断熱層内熱橋部分が「縦下地」））
                                       'Type'が'wall'または'BoundaryWall'で、かつ'InputMethod_wood'が
                                       'InputAllLayers'の場合以外は無視される。
        'CeilingConstructionMethod': # 天井の工法種類 string値, 以下のいずれか
                                     # 'Normal'（桁・梁間に断熱）
                                       'Type'が'BoundaryCeiling'で、かつ'InputMethod_wood'が
                                       'InputAllLayers'の場合以外は無視される。
        'RoofConstructionMethod'   : # 屋根の工法種類 string値, 以下のいずれか）,
                                     # 'Normal'（たるき間に断熱）,
                                     # 'Horizontal'（たるき間に断熱＋付加断熱（横下地））
                                       'Type'が'ceiling'で、かつ'InputMethod_wood'が
                                       'InputAllLayers'の場合以外は無視される。
        'Ur_wood'                  : # 補正熱貫流率（木造）(W/(m2・K))
                                       'Structure'が'wood'で、かつ'InputMethod_wood'が
                                       'InputUr'の場合以外は無視される。
        'Ur_steel'                 : # 補正熱貫流率（鉄骨造）(W/(m2・K))
                                       'Structure'が'steel'で、かつ'InputMethod_steel'が
                                       'InputUr'の場合以外は無視される。
        'Parts' : [
            {
                'AreaRatio'        : # 部位/面積比率（-）
                                       'InputMethod_wood'が'InputAllDetails'の場合以外は無視される。
                'Type_floor'       : # 部位/種別（床） string値,　以下のいずれか
                                     # 'Insulation'（断熱部分）,
                                     # 'Heatbridge'（熱橋部分）,
                                     # '（検討中）'（断熱部分（根太間断熱材＋大引間断熱材））,
                                     # '（検討中）'（断熱部分＋熱橋部分（根太間断熱材＋大引材等））,
                                     # '（検討中）'（断熱部分＋熱橋部分（根太材＋大引間断熱材））,
                                     # '（検討中）'（熱橋部分（根太材＋大引材等））
                                       'Type'が'floor'または'BoundaryFloor'で、かつ'InputMethod_wood'が
                                       'InputAllLayers'の場合以外は無視される。
                'Type_wall'        : # 部位/種別（壁） string値,　以下のいずれか
                                     # 'Insulation'（断熱部分）,
                                     # 'Heatbridge'（熱橋部分）,
                                     # '（検討中）'（断熱部分（柱・間柱間断熱材＋付加断熱材））,
                                     # '（検討中）'（断熱部分＋熱橋部分（柱・間柱間断熱材＋付加断熱層内熱橋部分））,
                                     # '（検討中）'（断熱部分＋熱橋部分（構造部材等＋付加断熱材））,
                                     # '（検討中）'（熱橋部分（構造部材等＋付加断熱層内熱橋部分））,
                                     # '（検討中）'（断熱部分（充填断熱材＋付加断熱材））,
                                     # '（検討中）'（断熱部分＋熱橋部（充填断熱材＋付加断熱層内熱橋部））,
                                     # '（検討中）'（断熱部分＋熱橋部（構造部材等＋付加断熱材））,
                                     # '（検討中）'（断熱部分＋熱橋部（まぐさ＋付加断熱材））,
                                     # '（検討中）'（熱橋部分（構造部材等＋付加断熱層内熱橋部））,
                                     # '（検討中）'（熱橋部分（まぐさ＋付加断熱層内熱橋部））
                                       'Type'が'wall'または'BoundaryWall'で、かつ'InputMethod_wood'が
                                       'InputAllLayers'の場合以外は無視される。
                'Type_ceiling'     : # 部位/種別（天井）	 string値,　以下のいずれか
                                     # 'Insulation'（断熱部分）,
                                     # 'Heatbridge'（熱橋部分）
                                       'Type'が'BoundaryCeiling'で、かつ'InputMethod_wood'が
                                       'InputAllLayers'の場合以外は無視される。
                'Type_roof'        : # 部位/種別（屋根） string値,　以下のいずれか
                                     # 'Insulation'（断熱部分）,
                                     # 'Heatbridge'（熱橋部分）,
                                     # '（検討中）'（断熱部分（たる木間断熱材＋付加断熱材））,
                                     # '（検討中）'（断熱部分＋熱橋部分（たる木間断熱材＋付加断熱層内熱橋部（下地たる木）））,
                                     # '（検討中）'（断熱部分＋熱橋部分（構造部材＋付加断熱材））,
                                     # '（検討中）'（熱橋部分（構造部材＋付加断熱層内熱橋部（下地たる木）））
                                       'Type'が'ceiling'で、かつ'InputMethod_wood'が
                                       'InputAllLayers'の場合以外は無視される。
                layers : [
                    {
                        'Name'     : # 部材名称　string値
                        'Thick'    : # 厚さ(m)
                        'Cond'     : # 熱伝導率(W/(m・K))
                        'Spech'    : # 容積比熱(J/(m3・K))
                    }
                ] # 層構成の情報
        ] # 部位の情報
        'IsSunshadeInput'          : # 日除けを入力するかどうか, bool値
                                       'IsSimplifiedInput'がTrueの場合は無視される。
        'y2'                       : # 開口部高さ(m)
                                       'IsSunshadeInput'がFalseの場合は無視される。
        'z'                        : # 出巾(m)
                                       'IsSunshadeInput'がFalseの場合は無視される。
        'y1'                       : # 窓上端と外部日除けの距離(m)
                                       'IsSunshadeInput'がFalseの場合は無視される。
    }]
}
```

#### Output

```
'Walls' : [
{
    'Name'                        : # 壁体名称
    'Structure'                   : # 一般部位構造種別 string値
    'Type'                        : # 壁体種別 string値
    'InputMethod_wood'            : # 壁体仕様入力方法（木造） string値
                                      'IsSimplifiedInput'がFalseかつStructure'が'wood'の場合のみ
    'InputMethod_rc'              : # 壁体仕様入力方法（鉄筋コンクリート造） string値
                                      'IsSimplifiedInput'がFalseかつStructure'が'rc'の場合のみ
    'InputMethod_steel'           : # 壁体仕様入力方法（鉄骨造） string値
                                      'IsSimplifiedInput'がFalseかつStructure'が'steel'の場合のみ
    'Direction'                   : # 方位　string値
    'Area'                        : # 部位面積(m2)
    'Space'                       : # 壁体が属する空間種類 string値, 以下のいずれか
                                    # 'main'（主たる居室）,
                                    # 'other'（その他居室）,
                                    # 'nonliving'（非居室）,
                                    # 'underfloor'（床下空間）
    'Ua'                          : # 平均熱貫流率(W/(m2・K))
                                      'InputMethod_wood'/'InputMethod_rc'/'InputMethod_steel'
                                      のいずれが'InputUa'の場合のみ
    'FloorConstructionMethod'     : # 床の工法種類 string値
                                      'Type'が'floor'または'BoundaryFloor'で、
                                      かつ'InputMethod_wood'が'InputAllLayers'の場合のみ
    'WallConstructionMethod'      : # 壁の工法種類 string値
                                      'Type'が'wall'または'BoundaryWall'で、
                                      かつ'InputMethod_wood'が'InputAllLayers'の場合のみ
    'CeilingConstructionMethod'   : # 天井の工法種類 string値）
                                      'Type'が'BoundaryCeiling'で、
                                      かつ'InputMethod_wood'が'InputAllLayers'の場合のみ
    'RoofConstructionMethod'      : # 屋根の工法種類 string値
                                      'Type'が'ceiling'で、
                                      かつ'InputMethod_wood'が'InputAllLayers'の場合のみ
    'Ur_wood'                     : # 補正熱貫流率（木造）(W/(m2・K))
                                      'Structure'が'wood'で、かつ'InputMethod_wood'が'InputUr'の場合のみ
    'Ur_steel'                    : # 補正熱貫流率（鉄骨造）(W/(m2・K))
                                      'Structure'が'steel'で、かつ'InputMethod_steel'が'InputUr'の場合のみ
    'Parts' : [
        {
            'AreaRatio'           : # 部位/面積比率（-）
                                      'InputMethod_wood'が'InputAllDetails'の場合のみ
            'Type_floor'          : # 部位/種別（床） string値
                                      'Type'が'floor'または'BoundaryFloor'で、
                                      かつ'InputMethod_wood'が'InputAllLayers'の場合のみ
            'Type_wall'           : # 部位/種別（壁） string値
                                      'Type'が'wall'または'BoundaryWall'で、
                                      かつ'InputMethod_wood'が'InputAllLayers'の場合のみ
            'Type_ceiling'        : # 部位/種別（天井）	string値
                                      'Type'が'BoundaryCeiling'で、
                                      かつ'InputMethod_wood'が'InputAllLayers'の場合のみ
            'Type_roof'           : # 部位/種別（屋根） string値
                                      'Type'が'ceiling'で、
                                      かつ'InputMethod_wood'が'InputAllLayers'の場合のみ
            layers : [
                {
                    'Name'        : # 部材名称　string値
                    'Thick'       : # 厚さ(m)
                    'Cond'        : # 熱伝導率(W/(m・K))
                    'Spech'       : # 容積比熱(J/(m3・K))
                }
            ] # 層構成の情報
            'IsSunshadeInput'     : # 日除けを入力するかどうか, bool値
                                      'IsSimplifiedInput'がFalseの場合のみ
            'y2'                  : # 開口部高さ(m)
                                      'IsSunshadeInput'がTrueの場合のみ
            'z'                   : # 出巾(m)
                                      'IsSunshadeInput'がTrueの場合のみ
            'y1'                  : # 窓上端と外部日除けの距離(m)
                                      'IsSunshadeInput'がTrueの場合のみ
        ] # 部位の情報
}
]
```

#### Function

In [6]:
def convert_wall( d ):
    
    # 壁体が属する空間種類の配列を作成 （'underfloor'（床下空間）はこの段階では不要？）
    spaces = ['main', 'other', 'nonliving']
    
    walls =[]
    for arg_wall in d['Walls']:
        for space in spaces:
            # 壁体名称、部位面積を更新し、「壁体が属する空間種類」を追加する
            wall = copy.deepcopy(arg_wall)
            wall['Name'] = arg_wall['Name'] + '_' + space, 
            wall['Area'] = GetAreaByRoomUse( arg_wall['Area'], d['MainOccupantRoomFloorArea'], d['OtherOccupantRoomFloorArea'], d['TotalFloorArea'], space ),
            wall['Space'] = space
            walls.append(wall)
        
    return { 'Walls' : walls }

#### Example

In [7]:
d = {
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0,
    'Walls': [
        {
            'Area': 67.8,
            'Direction': 'top',
            'Name': 'Ceiling',
            'Parts': [{
                    'AreaRatio': 1.0,
                    'Layers': [{'Cond': 0.045, 'Name': 'GW16K', 'Spech': 13.0, 'Thick': 0.0},
                               {'Cond': 0.22, 'Name': 'GPB', 'Spech': 830.0, 'Thick': 0.0095}]
                }],
            'Structure': 'wood',
            'Type': 'ceiling'
        },
        {
            'Area': 40.63,
            'Direction': 'SW',
            'Name': 'Wall_SW',
            'Parts': [{
                    'AreaRatio': 1.0,
                    'Layers': [{'Cond': 0.045, 'Name': 'GW16K', 'Spech': 13.0, 'Thick': 0.0},
                               {'Cond': 0.22, 'Name': 'GPB', 'Spech': 830.0, 'Thick': 0.0095}]
                }],
            'Structure': 'wood',
            'Type': 'wall'
        }
    ]
}    

convert_wall(d)

{'Walls': [{'Area': (16.95,),
   'Direction': 'top',
   'Name': ('Ceiling_main',),
   'Parts': [{'AreaRatio': 1.0,
     'Layers': [{'Cond': 0.045, 'Name': 'GW16K', 'Spech': 13.0, 'Thick': 0.0},
      {'Cond': 0.22, 'Name': 'GPB', 'Spech': 830.0, 'Thick': 0.0095}]}],
   'Space': 'main',
   'Structure': 'wood',
   'Type': 'ceiling'},
  {'Area': (33.9,),
   'Direction': 'top',
   'Name': ('Ceiling_other',),
   'Parts': [{'AreaRatio': 1.0,
     'Layers': [{'Cond': 0.045, 'Name': 'GW16K', 'Spech': 13.0, 'Thick': 0.0},
      {'Cond': 0.22, 'Name': 'GPB', 'Spech': 830.0, 'Thick': 0.0095}]}],
   'Space': 'other',
   'Structure': 'wood',
   'Type': 'ceiling'},
  {'Area': (16.95,),
   'Direction': 'top',
   'Name': ('Ceiling_nonliving',),
   'Parts': [{'AreaRatio': 1.0,
     'Layers': [{'Cond': 0.045, 'Name': 'GW16K', 'Spech': 13.0, 'Thick': 0.0},
      {'Cond': 0.22, 'Name': 'GPB', 'Spech': 830.0, 'Thick': 0.0095}]}],
   'Space': 'nonliving',
   'Structure': 'wood',
   'Type': 'ceiling'},
  {'A

### 3) 開口部の透明部位

- 開口部名称、方位、日射熱取得率[-]、熱貫流率[W/(m<sup>2</sup>・K)]などの情報は維持する。
- 「開口部が属する空間種類」の情報を追加し、部位面積から各部位の室用途別面積を推定する。

#### Input

```
{
    'MainOccupantRoomFloorArea'    : # 主たる居室の床面積(m2),
    'OtherOccupantRoomFloorArea'   : # その他の居室の床面積(m2),
    'TotalFloorArea'               : # 床面積の合計(m2),
    'Windows' : [
    {
        'Name'                     : # 開口部名称
        'Direction'                : # 方位　string値, 以下のいずれか
                                     # 'top', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'bottom',
                                     # 'ClosedSpace'（外気に通じていない空間）,
                                     # 'OpenBackFloor'（外気に通じる床裏）,
                                     # 'ResidenceSpace'（住戸及び住戸と同様の熱的環境の空間）,
                                     # 'ClosedBackFloor'（外気に通じていない床裏）,
                                     # 'EarthFloor'（土間床等）
        'Area'                     : # 部位面積(m2)
        'Uw'                       : # 窓の熱貫流率(W/(m2・K))
        'Type_window'              : # 窓の種類 string値, 'single' or 'double'
        
        # 'IsSimplifiedInput'がFalseの場合
        'IsInputEtaValue'          : # 日射熱取得率の入力方法 string値, 'InputValue' or 'InputSpec'
        
        # 'IsSimplifiedInput'がTrueの場合
        'EtaCooling'               : # 日射熱取得率（夏期）(-)
        'EtaHeating'               : # 日射熱取得率（冬期）(-)
        
        # 'Type_window'が'single'かつ'IsInputEtaValue'がTrueの場合
        'Eta'                      : # 日射熱取得率(-)
        
        # 'Type_window'が'single'かつ'IsInputEtaValue'がFalseの場合
        'Type_frame'               : # 窓枠種類 string値
        'Type_glass'               : # ガラス種類 string値
        'Type_shade'               : # 窓付属部材の種類 string値
        
        # 'Type_window'が'double'かつ'IsInputEtaValue'がTrueの場合
        'Eta_inside'               : # 日射熱取得率（室内側）(-)
        'Eta_outside'              : # 日射熱取得率（室外側）(-)
        
        # 'Type_window'が'double'かつ'IsInputEtaValue'がFalseの場合
        'Type_frame_inside'        : # 窓枠種類（室内側） string値
        'Type_glass_inside'        : # ガラス種類（室内側） string値
        'Type_shade_inside'        : # 窓付属部材の種類（室内側） string値
        'Type_frame_outside'       : # 窓枠種類（室外側） string値
        'Type_glass_outside'       : # ガラス種類（室外側） string値
        'Type_shade_outside'       : # 窓付属部材の種類（室外側） string値
        
        'IsSunshadeInput'          : # 日除けを入力するかどうか, bool値
                                       'IsSimplifiedInput'がTrueの場合は無視される。
                                       
        # 'IsSunshadeInput'がTrueの場合
        'y2'                       : # 開口部高さ(m)
        'z'                        : # 出巾(m)
        'y1'                       : # 窓上端と外部日除けの距離(m)
        
    }]
}
```

#### Output

```
'Windows' : [
{
    'Name'                     : # 開口部名称
    'Direction'                : # 方位　string値
    'Area'                     : # 部位面積(m2)
    'Space'                    : # 壁体が属する空間種類 string値, 以下のいずれか
                                 # 'main'（主たる居室）,
                                 # 'other'（その他居室）,
                                 # 'nonliving'（非居室）,
                                 # 'underfloor'（床下空間）
    'Uw'                       : # 窓の熱貫流率(W/(m2・K))
    'Type_window'              : # 窓の種類 string値, 'single' or 'double'
        
    # 'IsSimplifiedInput'がFalseの場合
    'IsInputEtaValue'          : # 日射熱取得率の入力方法 string値, 'InputValue' or 'InputSpec'
        
    # 'IsSimplifiedInput'がTrueの場合
    'EtaCooling'               : # 日射熱取得率（夏期）(-)
    'EtaHeating'               : # 日射熱取得率（冬期）(-)
        
    # 'Type_window'が'single'かつ'IsInputEtaValue'がTrueの場合
    'Eta'                      : # 日射熱取得率(-)
        
    # 'Type_window'が'single'かつ'IsInputEtaValue'がFalseの場合
    'Type_frame'               : # 窓枠種類 string値
    'Type_glass'               : # ガラス種類 string値
    'Type_shade'               : # 窓付属部材の種類 string値
        
    # 'Type_window'が'double'かつ'IsInputEtaValue'がTrueの場合
    'Eta_inside'               : # 日射熱取得率（室内側）(-)
    'Eta_outside'              : # 日射熱取得率（室外側）(-)
        
    # 'Type_window'が'double'かつ'IsInputEtaValue'がFalseの場合
    'Type_frame_inside'        : # 窓枠種類（室内側） string値
    'Type_glass_inside'        : # ガラス種類（室内側） string値
    'Type_shade_inside'        : # 窓付属部材の種類（室内側） string値
    'Type_frame_outside'       : # 窓枠種類（室外側） string値
    'Type_glass_outside'       : # ガラス種類（室外側） string値
    'Type_shade_outside'       : # 窓付属部材の種類（室外側） string値
        
    'IsSunshadeInput'          : # 日除けを入力するかどうか, bool値
                                   'IsSimplifiedInput'がTrueの場合は無視される。
                                       
    # 'IsSunshadeInput'がTrueの場合
    'y2'                       : # 開口部高さ(m)
    'z'                        : # 出巾(m)
    'y1'                       : # 窓上端と外部日除けの距離(m)   
}
]
```

#### Function

In [8]:
def convert_window( d ):
    
    # 開口部が属する空間種類の配列を作成
    spaces = ['main', 'other', 'nonliving']
    
    windows =[]
    for arg_window in d['Windows']:
        for space in spaces:
            # 開口部名称、部位面積を更新し、「開口部が属する空間種類」を追加する
            window = copy.deepcopy(arg_window)
            window['Name'] = arg_window['Name'] + '_' + space, 
            window['Area'] = GetAreaByRoomUse( arg_window['Area'], d['MainOccupantRoomFloorArea'], d['OtherOccupantRoomFloorArea'], d['TotalFloorArea'], space ),
            window['Space'] = space
            windows.append(window)
        
    return { 'Windows' : windows }

#### Example

In [9]:
d = {
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0,
    'Windows': [
        {
            'Name': 'WindowSW',
            'Direction': 'SW',
            'Area': 30.25,
            'Uw': 3.49,
            'Type_window': 'single',
            'IsInputEtaValue': 'InputValue',
            'Eta': 0.51,
            'IsSunshadeInput': False
        },
        {
            'Name': 'WindowNW',
            'Direction': 'NW',
            'Area': 3.17,
            'Uw': 3.49,
            'Type_window': 'single',
            'IsInputEtaValue': 'InputValue',
            'Eta': 0.51,
            'IsSunshadeInput': False
        }
    ]
}    

convert_window(d)

{'Windows': [{'Area': (7.5625,),
   'Direction': 'SW',
   'Eta': 0.51,
   'IsInputEtaValue': 'InputValue',
   'IsSunshadeInput': False,
   'Name': ('WindowSW_main',),
   'Space': 'main',
   'Type_window': 'single',
   'Uw': 3.49},
  {'Area': (15.125,),
   'Direction': 'SW',
   'Eta': 0.51,
   'IsInputEtaValue': 'InputValue',
   'IsSunshadeInput': False,
   'Name': ('WindowSW_other',),
   'Space': 'other',
   'Type_window': 'single',
   'Uw': 3.49},
  {'Area': (7.5625,),
   'Direction': 'SW',
   'Eta': 0.51,
   'IsInputEtaValue': 'InputValue',
   'IsSunshadeInput': False,
   'Name': ('WindowSW_nonliving',),
   'Space': 'nonliving',
   'Type_window': 'single',
   'Uw': 3.49},
  {'Area': (0.7925,),
   'Direction': 'NW',
   'Eta': 0.51,
   'IsInputEtaValue': 'InputValue',
   'IsSunshadeInput': False,
   'Name': ('WindowNW_main',),
   'Space': 'main',
   'Type_window': 'single',
   'Uw': 3.49},
  {'Area': (1.585,),
   'Direction': 'NW',
   'Eta': 0.51,
   'IsInputEtaValue': 'InputValue',
  

#### 4) 開口部の不透明部位

- Lv2の情報を維持する。

#### Input

```
{
    'Doors' : [
    {
        'Name'                     : # 開口部名称
        'Direction'                : # 方位　string値, 以下のいずれか
                                     # 'top', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'bottom',
                                     # 'ClosedSpace'（外気に通じていない空間）,
                                     # 'OpenBackFloor'（外気に通じる床裏）,
                                     # 'ResidenceSpace'（住戸及び住戸と同様の熱的環境の空間）,
                                     # 'ClosedBackFloor'（外気に通じていない床裏）,
                                     # 'EarthFloor'（土間床等）
        'Area'                     : # 部位面積(m2)
        'U'                        : # 熱貫流率(W/(m2・K))
        'IsSunshadeInput'          : # 日除けを入力するかどうか, bool値
                                       'IsSimplifiedInput'がTrueの場合は無視される。
                                       
        # 'IsSunshadeInput'がTrueの場合
        'y2'                       : # 開口部高さ(m)
        'z'                        : # 出巾(m)
        'y1'                       : # 窓上端と外部日除けの距離(m)
    }]
}
```

#### Output

```
'Windows' : [
{
    'Name'                     : # 開口部名称
    'Direction'                : # 方位　string値
    'Area'                     : # 部位面積(m2)
    'U'                        : # 熱貫流率(W/(m2・K))
    'IsSunshadeInput'          : # 日除けを入力するかどうか, bool値
                                   'IsSimplifiedInput'がTrueの場合は無視される。
                                       
    # 'IsSunshadeInput'がTrueの場合
    'y2'                       : # 開口部高さ(m)
    'z'                        : # 出巾(m)
    'y1'                       : # 窓上端と外部日除けの距離(m)   
}
]
```

#### Function

In [10]:
def convert_door( d ):
    doors = copy.deepcopy(d['Doors'])
    
    return { 'Doors' : doors }

#### Example

In [11]:
d = {
    'Doors': [
        {
            'Name': 'DoorNW',
            'Direction': 'NW',
            'Area': 52,
            'U': 2.33,
            'IsSunshadeInput': False
        }
    ]
}    
convert_door(d)

{'Doors': [{'Area': 52,
   'Direction': 'NW',
   'IsSunshadeInput': False,
   'Name': 'DoorNW',
   'U': 2.33}]}

## 2-5. コンバート【分類：部位情報】

- 壁体・開口部名称、外皮フラグ、日射フラグ、日射フラグ、非定常フラグ、外部日除け名称の情報は維持する。
- 新たに変数「空間種類」「隣接空間種類」を作成する。
- 用途別の部位面積[m<sup>2</sup>]、部位長さ[m<sup>2</sup>]を、室用途別の床面積より推定する。
- 土間床等面積が0㎡より大きい場合、基礎断熱部は床下空間に属するものとみなし、室と床下空間の間の床の合計面積は土間床等面積の合計に等しいとみなす。室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）

### Input

```
data = {
    'name'                        : # 壁体・開口部名称
    'FlagPerimeter'               : # 外皮フラグ 1 ~ 2 の整数
    'FlagSolarRadiation'          : # 日射フラグ 1 ~ 2 の整数
    'direction'                   : # 方位 1 ~ 14 の整数
    'FlagUnsteady'                : # 非定常フラグ 1 ~ 2 の整数
    'A'                           : # 部位面積(m2)
    'L'                           : # 部位長さ(m)
    'sunshade'                    : # 外部日除け名称
}
```

### Output

```
data = {
    'name'                        : # 壁体・開口部名称
    'Room'                        : # 空間種類
    'FlagPerimeter'               : # 外皮フラグ 1 ~ 2 の整数
    'FlagSolarRadiation'          : # 日射フラグ 1 ~ 2 の整数
    'direction'                   : # 方位 1 ~ 14 の整数
    'AdjacentSpace'               : # 隣接空間種類
    'FlagUnsteady'                : # 非定常フラグ 1 ~ 2 の整数
    'A'                           : # 部位面積(m2)
    'L'                           : # 部位長さ(m)
    'sunshade'                    : # 外部日除け名称
}
```

### Function

In [9]:
def convert_element(d_common,d_elements):

    for d_element in d_elements:
        d_element['AdjacentSpace'] = None
    AEarthFloor=0
    for d in d_elements:
        if d['direction']==15:
            AEarthFloor=d['A']        
    if AEarthFloor>0:
        d_elements.append({'name': 'FloorNotPerimeter','FlagPerimeter': 1,'FlagSolarRadiation': 2,'direction': 14,
                       'AdjacentSpace':'Underfloor','FlagUnsteady': 1,'A':AEarthFloor,'L': None,'sunshade': None})
            
    #主たる居室、その他の居室、非居室の床面積比率
    rMain=d_common['MainOccupantRoomFloorArea']/d_common['TotalFloorArea']
    rOther=d_common['OtherOccupantRoomFloorArea']/d_common['TotalFloorArea']
    rNon=1-rMain-rOther
    
    elements=[]
    for d_element in d_elements:
        if d_element['direction']==15:
            elements.append({
                'name'                        : d_element['name'],
                'Room'                        : 'Underfloor',
                'FlagPerimeter'               : d_element['FlagPerimeter'],
                'FlagSolarRadiation'          : d_element['FlagSolarRadiation'],
                'direction'                   : d_element['direction'],
                'AdjacentSpace'               : d_element['AdjacentSpace'],
                'FlagUnsteady'                : d_element['FlagUnsteady'],
                'A'                           : d_element['A'],
                'L'                           : d_element['L'],
                'sunshade'                    : d_element['sunshade']
            })
        else:
            if d_element['A'] is not None:
                AMain=d_element['A']*rMain
                AOther=d_element['A']*rOther
                ANon=d_element['A']*rNon
            else:
                AMain=None
                AOther=None
                ANon=d_None       

            if d_element['L'] is not None:
                LMain=d_element['L']*rMain
                LOther=d_element['L']*rOther
                LNon=d_element['L']*rNon
            else:
                LMain=None
                LOther=None
                LNon=None 

            temporary=[{
                'name'                        : d_element['name'],
                'Room'                        : 'MainOccupantRoom',
                'FlagPerimeter'               : d_element['FlagPerimeter'],
                'FlagSolarRadiation'          : d_element['FlagSolarRadiation'],
                'direction'                   : d_element['direction'],
                'AdjacentSpace'               : d_element['AdjacentSpace'],
                'FlagUnsteady'                : d_element['FlagUnsteady'],
                'A'                           : AMain,
                'L'                           : LMain,
                'sunshade'                    : d_element['sunshade']
            },{
                'name'                        : d_element['name'],
                'Room'                        : 'OtherOccupantRoomFloorArea',
                'FlagPerimeter'               : d_element['FlagPerimeter'],
                'FlagSolarRadiation'          : d_element['FlagSolarRadiation'],
                'direction'                   : d_element['direction'],
                'AdjacentSpace'               : d_element['AdjacentSpace'],
                'FlagUnsteady'                : d_element['FlagUnsteady'],
                'A'                           : AOther,
                'L'                           : LOther,
                'sunshade'                    : d_element['sunshade']
            },{
                'name'                        : d_element['name'],
                'Room'                        : 'NonOccupantRoomFloorArea',
                'FlagPerimeter'               : d_element['FlagPerimeter'],
                'FlagSolarRadiation'          : d_element['FlagSolarRadiation'],
                'direction'                   : d_element['direction'],
                'AdjacentSpace'               : d_element['AdjacentSpace'],
                'FlagUnsteady'                : d_element['FlagUnsteady'],
                'A'                           : ANon,
                'L'                           : LNon,
                'sunshade'                    : d_element['sunshade']
            }]
            for i in range(0,3):
                elements.append(temporary[i])

    return elements

### Example

In [10]:
convert_element({
    'Region' : 6,
    'FlagSimplifiedEnvelopeInput': 1,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0
    },[{
            'name'                    : 'Floor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 12,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'EarthFloor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 15,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        }
    ]
)

[{'A': 7.5,
  'AdjacentSpace': None,
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'MainOccupantRoom',
  'direction': 12,
  'name': 'Floor',
  'sunshade': None},
 {'A': 15.0,
  'AdjacentSpace': None,
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'OtherOccupantRoomFloorArea',
  'direction': 12,
  'name': 'Floor',
  'sunshade': None},
 {'A': 7.5,
  'AdjacentSpace': None,
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'NonOccupantRoomFloorArea',
  'direction': 12,
  'name': 'Floor',
  'sunshade': None},
 {'A': 30,
  'AdjacentSpace': None,
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'Underfloor',
  'direction': 15,
  'name': 'EarthFloor',
  'sunshade': None},
 {'A': 7.5,
  'AdjacentSpace': 'Underfloor',
  'FlagPerimeter': 1,
  'FlagSolarRadiation': 2,
  'FlagUnsteady': 1,
  'L': None,
  'Room': 'MainOccu

## 2-6. コンバート【全分類】

In [11]:
def Convert(d):
    d_common=convert_common(d['Common'])
    d_ceiling=convert_wall(d['Ceiling'])
    d_wall=convert_wall(d['Wall'])
    d_floor=convert_wall(d['Floor'])
    d_window=convert_window(d['Window'])
    d_door=convert_window(d['Door'])
    d_sunshade=convert_sunshade(d['Sunshade'])

    temporary=convert_element(d_common, d['Elements'])
    d_element=[]
    for e in temporary:
        if e['name']=='Ceiling':
            e['Wall']=d_ceiling
        elif e['name']=='Wall':
            e['Wall']=d_wall
        elif e['name']=='Floor':
            e['Wall']=d_floor
        elif e['name']=='EarthFloor':
            e['FloorNonPerimeter']={
                    'name'                       : 'FloorNotPerimeter',
                    'structure'                  : 1,
                    'type'                       : 3,
                    'Layers'                     : [{'material': 'PED','d': 0.012,'rambda': 0.16,'Crho': 720}],
                    'Ua'                         : 3.77,
                    'Ug'                         : None,
                    'HeatBridge'                 : None,
                    'rHeatBridge'                : None,
                    'UHeatBridge'                : None,
                    'lHeatBridge'                : None,
                    'psiHeatBridge'              : None,
                    'Ur'                         : None
                }
        elif e['name']=='Window':
            e['Window']=d_window
            e['Sunshade']=d_sunshade
        elif e['name']=='Door':
            e['Door']=d_door
        d_element.append(e)

    return {'common':d_common,'element':d_element}

In [12]:
d = {
    'Common' : {
        'Region'                     : 6,
        'FlagSimplifiedEnvelopeInput': 2,
        'MainOccupantRoomFloorArea'  : 30.0,
        'OtherOccupantRoomFloorArea' : 60.0,
        'TotalFloorArea'             : 120.0
    },
    'Ceiling' : {
        'name'                       : 'Ceiling',
        'structure'                  : 1,
        'type'                       : 1,
        'Layers'                     : [{'material': 'GW16K','d': 0.179,'rambda': 0.045,'Crho': 13},
                                        {'material': 'GPB','d': 0.0095,'rambda': 0.22,'Crho': 830}],
        'Ua'                         : 0.24,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Wall' : {
        'name'                       : 'Wall',
        'structure'                  : 1,
        'type'                       : 2,
        'Layers'                     : [{'material': 'GW16K','d': 0.076,'rambda': 0.045,'Crho': 13},
                                        {'material': 'GPB','d': 0.0095,'rambda': 0.22,'Crho': 830}],
        'Ua'                         : 0.53,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Floor' : {
        'name'                       : 'Floor',
        'structure'                  : 1,
        'type'                       : 3,
        'Layers'                     : [{'material': 'GW16K','d': 0.081,'rambda': 0.045,'Crho': 13},
                                        {'material': 'PED','d': 0.012,'rambda': 0.16,'Crho': 720}],
        'Ua'                         : 0.48,
        'Ug'                         : None,
        'HeatBridge'                 : None,
        'rHeatBridge'                : None,
        'UHeatBridge'                : None,
        'lHeatBridge'                : None,
        'psiHeatBridge'              : None,
        'Ur'                         : None
    },
    'Window' : {
        'name'                        : 'Window',
        'eta'                         : 0.738,
        'etaCooling'                  : None,
        'etaHeating'                  : None,
        'U'                           : 4.65
    },
    'Door' : {
        'name'                        : 'Door',
        'eta'                         : round(2.33*0.034,3),
        'etaCooling'                  : None,
        'etaHeating'                  : None,
        'U'                           : 2.33
    },
    'Sunshade' : {
        'name'                        : 'Sunshade',
        'y1'                          : 0.5,
        'y2'                          : 2, 
        'z'                           : 1,
        'fCooling'                    : None,
        'fHeating'                    : None
    },
    'Elements' : [{
            'name'                    : 'Ceiling',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 1,
            'FlagUnsteady'            : 1,
            'A'                       : 60,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Wall',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 6,
            'FlagUnsteady'            : 1,
            'A'                       : 60,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Floor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 12,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'EarthFloor',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 2,
            'direction'               : 15,
            'FlagUnsteady'            : 1,
            'A'                       : 30,
            'L'                       : None,
            'sunshade'                : None
        },{
            'name'                    : 'Window',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 6,
            'FlagUnsteady'            : 2,
            'A'                       : 6,
            'L'                       : None,
            'sunshade'                : 'Sunshade'
        },{
            'name'                    : 'Door',
            'FlagPerimeter'           : 1,
            'FlagSolarRadiation'      : 1,
            'direction'               : 2,
            'FlagUnsteady'            : 2,
            'A'                       : 3,
            'L'                       : None,
            'sunshade'                : None
        }
    ]
}

Convert(d)

{'common': {'FlagSimplifiedEnvelopeInput': 2,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'element': [{'A': 15.0,
   'AdjacentSpace': None,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
   'FlagUnsteady': 1,
   'L': None,
   'Room': 'MainOccupantRoom',
   'Wall': {'HeatBridge': None,
    'Layers': [{'Crho': 13, 'd': 0.179, 'material': 'GW16K', 'rambda': 0.045},
     {'Crho': 830, 'd': 0.0095, 'material': 'GPB', 'rambda': 0.22}],
    'UHeatBridge': None,
    'Ua': 0.24,
    'Ug': None,
    'Ur': None,
    'lHeatBridge': None,
    'name': 'Ceiling',
    'psiHeatBridge': None,
    'rHeatBridge': None,
    'structure': 1,
    'type': 1},
   'direction': 1,
   'name': 'Ceiling',
   'sunshade': None},
  {'A': 30.0,
   'AdjacentSpace': None,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
   'FlagUnsteady': 1,
   'L': None,
   'Room': 'OtherOccupantRoomFloorArea',
   'Wall': {'HeatBridge': None,
    'Layers': [